In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ML').getOrCreate()

from pyspark.ml.regression import LinearRegression

In [2]:
df_training = spark.read.format("libsvm").load("../Arquivos/sample_linear_regression_data.txt")

In [3]:
df_training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [4]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [5]:
lrModel = lr.fit(df_training)

In [6]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [7]:
lrModel.intercept

0.14228558260358093

In [9]:
training_summary = lrModel.summary

In [10]:
training_summary.r2

0.027839179518600154

In [11]:
training_summary.rootMeanSquaredError

10.16309157133015

In [12]:
df_all_data = df_training

In [15]:
train_data, test_data = df_all_data.randomSplit([0.7, 0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                366|
|   mean|0.05846887574804751|
| stddev| 10.270413977559837|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                135|
|   mean| 0.7948273436733712|
| stddev|  10.46500124299265|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [19]:
correct_model = lr.fit(train_data)

In [20]:
test_results = correct_model.evaluate(test_data)

In [22]:
test_results.residuals.show();

+-------------------+
|          residuals|
+-------------------+
|-25.962550717349757|
|-22.949196082005518|
|-20.459662230078944|
| -19.91539986949981|
|-18.781564031722635|
| -20.39075624940786|
|-18.968368326186404|
|-18.676974186510392|
|  -20.0474024966176|
|-16.345284084958344|
|-10.952146943258356|
|-16.381844858125188|
|-16.371479294351335|
| -11.88521529353768|
|-16.351440578255875|
| -13.51049275176337|
|-14.061951738539902|
|-11.986348740120794|
|  -9.77641006613137|
| -9.648189231745771|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

10.79960654903111

In [24]:
unlabeled_data = test_data.select("features")

In [25]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [26]:
predictions = correct_model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -2.0834673204268777|
|(10,[0,1,2,3,4,5,...| -3.7870111005962053|
|(10,[0,1,2,3,4,5,...| -0.9727255340868608|
|(10,[0,1,2,3,4,5,...|  0.5130638392852594|
|(10,[0,1,2,3,4,5,...|  0.5063504657180041|
|(10,[0,1,2,3,4,5,...|   2.962081678468355|
|(10,[0,1,2,3,4,5,...|  1.9029687003103892|
|(10,[0,1,2,3,4,5,...|  2.5256248352332795|
|(10,[0,1,2,3,4,5,...|   4.266717463994301|
|(10,[0,1,2,3,4,5,...|  0.9857392051256649|
|(10,[0,1,2,3,4,5,...| -4.3967242121208985|
|(10,[0,1,2,3,4,5,...|  1.3253618835827543|
|(10,[0,1,2,3,4,5,...|  1.6087210414202073|
|(10,[0,1,2,3,4,5,...| -1.1547127705669342|
|(10,[0,1,2,3,4,5,...|   3.429217474885454|
|(10,[0,1,2,3,4,5,...|  1.0097189664083166|
|(10,[0,1,2,3,4,5,...|  1.5705096609934892|
|(10,[0,1,2,3,4,5,...|-0.49293147133070225|
|(10,[0,1,2,3,4,5,...| -2.6345363367087895|
|(10,[0,1,2,3,4,5,...|  -2.48216